In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import DenseVector  
from pyspark.ml.stat import Correlation
from pyspark.ml.stat import ChiSquareTest
from pyspark.mllib.linalg import Matrix, Matrices
from pyspark.mllib.stat import Statistics
from math import sqrt
import sys
import numpy as np 
import pandas as pd

In [12]:
def getItemPair(line):
    (item1, item2, reorders1, reorders2)=line.split(',')
    return ((int(item1), int(item2)), (float(reorders1), float(reorders2)))

def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1
    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)
    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))
        return (score, numPairs)

In [7]:
conf = SparkConf().setMaster("local[*]").setAppName("itembased")
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate(conf=conf)
sc

<SparkContext master=local[*] appName=pyspark-shell>

# Load data

In [8]:
data = sc.textFile('./reorder_itemPair_v3.csv')
#removes the header
header = data.first()
data = data.filter(lambda line:line if line!= header else False)
data.take(5)

['25,10,9,3', '25,10,11,13', '25,10,5,3', '25,10,4,5', '25,10,3,3']

# Get item pair ((item1, item2), (reorders1, reorders2))

In [9]:
#Make the product_id pairs as key
itemPairs = data.map(getItemPair)
itemPairGrouped = itemPairs.groupByKey()
itemPairGrouped.take(5)

[((45, 44), <pyspark.resultiterable.ResultIterable at 0x115e4f850>),
 ((184, 54), <pyspark.resultiterable.ResultIterable at 0x115e4f750>),
 ((204, 162), <pyspark.resultiterable.ResultIterable at 0x115e4fa10>),
 ((260, 94), <pyspark.resultiterable.ResultIterable at 0x115e4f950>),
 ((260, 141), <pyspark.resultiterable.ResultIterable at 0x115e4fa90>)]

In [10]:
itemPairs.take(5)

[((25, 10), (9.0, 3.0)),
 ((25, 10), (11.0, 13.0)),
 ((25, 10), (5.0, 3.0)),
 ((25, 10), (4.0, 5.0)),
 ((25, 10), (3.0, 3.0))]

# Measure the similarity of their orders across all user who bought both

In [13]:
#compute similarities between reorders for each item in pair by cosine similarity (score, numPairs)
itemPairSimilarities = itemPairGrouped.mapValues(computeCosineSimilarity).cache()
itemPairSimilarities.take(5)

[((45, 44), (0.9866952072393964, 3)),
 ((184, 54), (0.782061887005775, 3)),
 ((204, 162), (0.8012858730445609, 8)),
 ((260, 94), (0.6519253397700332, 12)),
 ((260, 141), (0.7596810692421552, 17))]

# Get the result

In [63]:
flat_pair = itemPairSimilarities.map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][1]))
spark = SparkSession(sc)
hasattr(flat_pair, "toDF")
result_df = flat_pair.toDF(['item_1', 'item_2', 'score', 'numPairs'])
result_df.show()


+------+------+------------------+--------+
|item_1|item_2|             score|numPairs|
+------+------+------------------+--------+
|    45|    10|0.8671471180970177|       6|
|   106|    45|0.5518039894046952|       4|
|   141|    93|               1.0|       2|
|   148|    45|0.8194104605060346|      43|
|   196|    28|0.8691638320240437|       3|
|   196|   184|0.8777082172458632|       7|
|   219|    45|0.8573562625631047|       3|
|   239|   130|0.9984603532054125|       2|
|   260|   130| 0.736449378529043|       3|
|   260|   169|0.9970544855015815|       2|
|   276|   196|0.9828140504849788|       5|
|   277|   196|0.9172018493262263|       9|
|   278|    45|               1.0|       2|
|   280|   196|0.9590084765982818|       7|
|   286|   248| 0.970142500145332|       2|
|   310|   117|0.9761870601839528|       2|
|   311|   160|0.9298547766256364|       3|
|   311|   260|0.6546775737165424|      51|
|   329|    34|0.7528958613625764|      10|
|   329|    94|0.749334172596375

In [70]:
result_df.describe(['score', 'numPairs']).show()

+-------+-------------------+------------------+
|summary|              score|          numPairs|
+-------+-------------------+------------------+
|  count|            6292110|           6292110|
|   mean| 0.8427283605839218| 8.195531387722085|
| stddev|0.14273413335694768|42.841218600697545|
|    min|0.09663130098844623|                 2|
|    max| 1.0000000000000002|             12034|
+-------+-------------------+------------------+



# Filtering result : score> 0.83 & coOccurence > 8

In [119]:
scoreThreshold = 0.83
coOccurenceThreshold = 8
filterResults = itemPairSimilarities.filter(lambda x:x if x[1][0] > scoreThreshold and x[1][1] > coOccurenceThreshold else False)
filterResults.take(5)


[((277, 196), (0.9172018493262263, 9)),
 ((890, 691), (0.8916907184704714, 10)),
 ((1090, 1025), (0.8819999115995705, 16)),
 ((1234, 651), (0.8851679774100251, 9)),
 ((1263, 432), (0.874542061297446, 10))]

In [120]:
flat_pair = filterResults.map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][1]))
spark = SparkSession(sc)
hasattr(flat_pair, "toDF")
result_df = flat_pair.toDF(['item_1', 'item_2', 'score', 'numPairs'])
result_df.show()

+------+------+------------------+--------+
|item_1|item_2|             score|numPairs|
+------+------+------------------+--------+
|   277|   196|0.9172018493262263|       9|
|   890|   691|0.8916907184704714|      10|
|  1090|  1025|0.8819999115995705|      16|
|  1234|   651|0.8851679774100251|       9|
|  1263|   432| 0.874542061297446|      10|
|  1376|  1025|0.8521910045269907|      12|
|  1443|   329| 0.857744259317666|      15|
|  1529|   117|0.8699554997426215|      13|
|  1940|   895|0.9228720068506988|      10|
|  2078|  1529|0.8482297532940771|      31|
|  2180|  2091|0.9122422587059412|      13|
|  2210|   365|0.8891934322651346|      10|
|  2228|  1090|0.8354425598906305|      11|
|  2295|  2238|0.8348668725161135|      10|
|  2326|  1090|0.8402906411319024|      24|
|  2361|   651|0.9786892119359873|       9|
|  2450|   329|0.9183886379720622|       9|
|  2450|  1700|0.9424252620630665|      15|
|  2581|    45|0.8723002548293942|      10|
|  2770|   287| 0.97253582284390

In [121]:
result_df.describe(['score', 'numPairs']).show()
result_df.cache()

+-------+-------------------+------------------+
|summary|              score|          numPairs|
+-------+-------------------+------------------+
|  count|             176216|            176216|
|   mean| 0.8797062410974129|14.235551822762973|
| stddev|0.03591802846519989|10.906560215166024|
|    min| 0.8300002777373985|                 9|
|    max| 0.9965359762835524|              1813|
+-------+-------------------+------------------+



DataFrame[item_1: bigint, item_2: bigint, score: double, numPairs: bigint]

# Create ranking csv

In [122]:
allitem_1 = [int(row['item_1']) for row in result_df.select('item_1').collect()]
allitem_2 = [int(row['item_2']) for row in result_df.select('item_2').collect()]
allitem = allitem_1 + allitem_2
productIds = np.unique(allitem).tolist()
print('product count: ' , len(productIds))

product count:  10827


In [124]:
ranking = ['rank_'+ str(i) for i in range(1,11)]
df = pd.DataFrame(np.zeros((10827, 10), dtype=np.int), index = productIds, columns = ranking)
df.index.name = 'product_id'
df.reset_index(level=0, inplace=True)
df

,product_id,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
0,1,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,0
2,10,0,0,0,0,0,0,0,0,0,0
3,23,0,0,0,0,0,0,0,0,0,0
4,25,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
10822,49655,0,0,0,0,0,0,0,0,0,0
10823,49667,0,0,0,0,0,0,0,0,0,0
10824,49668,0,0,0,0,0,0,0,0,0,0
10825,49680,0,0,0,0,0,0,0,0,0,0


In [125]:
all_id = [int(ids) for ids in df['product_id']]
count = 0
for ids in all_id:
    item_id = int(ids)
    rank = result_df.filter((result_df.item_1 == item_id) | (result_df.item_2 == item_id)).sort('score', ascending=False).limit(10)
    get_item_1 = rank.filter(rank.item_1 != item_id).select('item_1')
    get_item_2 = rank.filter(rank.item_2 != item_id).select('item_2')
    id_list_1 = [int(row['item_1']) for row in get_item_1.collect()]
    id_list_2 = [int(row['item_2']) for row in get_item_2.collect()]
    rank10 = id_list_1 + id_list_2
    for i in range(len(rank10)):
        value = rank10[i]
        i+=1
        col = 'rank_' + str(i)
        df.loc[df['product_id'] == item_id, col] = value
    
    count+=1
    print('insert %s record'%count)

df

insert 1 record
insert 2 record
insert 3 record
insert 4 record
insert 5 record
insert 6 record
insert 7 record
insert 8 record
insert 9 record
insert 10 record
insert 11 record
insert 12 record
insert 13 record
insert 14 record
insert 15 record
insert 16 record
insert 17 record
insert 18 record
insert 19 record
insert 20 record
insert 21 record
insert 22 record
insert 23 record
insert 24 record
insert 25 record
insert 26 record
insert 27 record
insert 28 record
insert 29 record
insert 30 record
insert 31 record
insert 32 record
insert 33 record
insert 34 record
insert 35 record
insert 36 record
insert 37 record
insert 38 record
insert 39 record
insert 40 record
insert 41 record
insert 42 record
insert 43 record
insert 44 record
insert 45 record
insert 46 record
insert 47 record
insert 48 record
insert 49 record
insert 50 record
insert 51 record
insert 52 record
insert 53 record
insert 54 record
insert 55 record
insert 56 record
insert 57 record
insert 58 record
insert 59 record
insert

insert 463 record
insert 464 record
insert 465 record
insert 466 record
insert 467 record
insert 468 record
insert 469 record
insert 470 record
insert 471 record
insert 472 record
insert 473 record
insert 474 record
insert 475 record
insert 476 record
insert 477 record
insert 478 record
insert 479 record
insert 480 record
insert 481 record
insert 482 record
insert 483 record
insert 484 record
insert 485 record
insert 486 record
insert 487 record
insert 488 record
insert 489 record
insert 490 record
insert 491 record
insert 492 record
insert 493 record
insert 494 record
insert 495 record
insert 496 record
insert 497 record
insert 498 record
insert 499 record
insert 500 record
insert 501 record
insert 502 record
insert 503 record
insert 504 record
insert 505 record
insert 506 record
insert 507 record
insert 508 record
insert 509 record
insert 510 record
insert 511 record
insert 512 record
insert 513 record
insert 514 record
insert 515 record
insert 516 record
insert 517 record
insert 518

insert 919 record
insert 920 record
insert 921 record
insert 922 record
insert 923 record
insert 924 record
insert 925 record
insert 926 record
insert 927 record
insert 928 record
insert 929 record
insert 930 record
insert 931 record
insert 932 record
insert 933 record
insert 934 record
insert 935 record
insert 936 record
insert 937 record
insert 938 record
insert 939 record
insert 940 record
insert 941 record
insert 942 record
insert 943 record
insert 944 record
insert 945 record
insert 946 record
insert 947 record
insert 948 record
insert 949 record
insert 950 record
insert 951 record
insert 952 record
insert 953 record
insert 954 record
insert 955 record
insert 956 record
insert 957 record
insert 958 record
insert 959 record
insert 960 record
insert 961 record
insert 962 record
insert 963 record
insert 964 record
insert 965 record
insert 966 record
insert 967 record
insert 968 record
insert 969 record
insert 970 record
insert 971 record
insert 972 record
insert 973 record
insert 974

insert 1355 record
insert 1356 record
insert 1357 record
insert 1358 record
insert 1359 record
insert 1360 record
insert 1361 record
insert 1362 record
insert 1363 record
insert 1364 record
insert 1365 record
insert 1366 record
insert 1367 record
insert 1368 record
insert 1369 record
insert 1370 record
insert 1371 record
insert 1372 record
insert 1373 record
insert 1374 record
insert 1375 record
insert 1376 record
insert 1377 record
insert 1378 record
insert 1379 record
insert 1380 record
insert 1381 record
insert 1382 record
insert 1383 record
insert 1384 record
insert 1385 record
insert 1386 record
insert 1387 record
insert 1388 record
insert 1389 record
insert 1390 record
insert 1391 record
insert 1392 record
insert 1393 record
insert 1394 record
insert 1395 record
insert 1396 record
insert 1397 record
insert 1398 record
insert 1399 record
insert 1400 record
insert 1401 record
insert 1402 record
insert 1403 record
insert 1404 record
insert 1405 record
insert 1406 record
insert 1407 

insert 1787 record
insert 1788 record
insert 1789 record
insert 1790 record
insert 1791 record
insert 1792 record
insert 1793 record
insert 1794 record
insert 1795 record
insert 1796 record
insert 1797 record
insert 1798 record
insert 1799 record
insert 1800 record
insert 1801 record
insert 1802 record
insert 1803 record
insert 1804 record
insert 1805 record
insert 1806 record
insert 1807 record
insert 1808 record
insert 1809 record
insert 1810 record
insert 1811 record
insert 1812 record
insert 1813 record
insert 1814 record
insert 1815 record
insert 1816 record
insert 1817 record
insert 1818 record
insert 1819 record
insert 1820 record
insert 1821 record
insert 1822 record
insert 1823 record
insert 1824 record
insert 1825 record
insert 1826 record
insert 1827 record
insert 1828 record
insert 1829 record
insert 1830 record
insert 1831 record
insert 1832 record
insert 1833 record
insert 1834 record
insert 1835 record
insert 1836 record
insert 1837 record
insert 1838 record
insert 1839 

...

insert 9142 record
insert 9143 record
insert 9144 record
insert 9145 record
insert 9146 record
insert 9147 record
insert 9148 record
insert 9149 record
insert 9150 record
insert 9151 record
insert 9152 record
insert 9153 record
insert 9154 record
insert 9155 record
insert 9156 record
insert 9157 record
insert 9158 record
insert 9159 record
insert 9160 record
insert 9161 record
insert 9162 record
insert 9163 record
insert 9164 record
insert 9165 record
insert 9166 record
insert 9167 record
insert 9168 record
insert 9169 record
insert 9170 record
insert 9171 record
insert 9172 record
insert 9173 record
insert 9174 record
insert 9175 record
insert 9176 record
insert 9177 record
insert 9178 record
insert 9179 record
insert 9180 record
insert 9181 record
insert 9182 record
insert 9183 record
insert 9184 record
insert 9185 record
insert 9186 record
insert 9187 record
insert 9188 record
insert 9189 record
insert 9190 record
insert 9191 record
insert 9192 record
insert 9193 record
insert 9194 

insert 9575 record
insert 9576 record
insert 9577 record
insert 9578 record
insert 9579 record
insert 9580 record
insert 9581 record
insert 9582 record
insert 9583 record
insert 9584 record
insert 9585 record
insert 9586 record
insert 9587 record
insert 9588 record
insert 9589 record
insert 9590 record
insert 9591 record
insert 9592 record
insert 9593 record
insert 9594 record
insert 9595 record
insert 9596 record
insert 9597 record
insert 9598 record
insert 9599 record
insert 9600 record
insert 9601 record
insert 9602 record
insert 9603 record
insert 9604 record
insert 9605 record
insert 9606 record
insert 9607 record
insert 9608 record
insert 9609 record
insert 9610 record
insert 9611 record
insert 9612 record
insert 9613 record
insert 9614 record
insert 9615 record
insert 9616 record
insert 9617 record
insert 9618 record
insert 9619 record
insert 9620 record
insert 9621 record
insert 9622 record
insert 9623 record
insert 9624 record
insert 9625 record
insert 9626 record
insert 9627 

insert 10007 record
insert 10008 record
insert 10009 record
insert 10010 record
insert 10011 record
insert 10012 record
insert 10013 record
insert 10014 record
insert 10015 record
insert 10016 record
insert 10017 record
insert 10018 record
insert 10019 record
insert 10020 record
insert 10021 record
insert 10022 record
insert 10023 record
insert 10024 record
insert 10025 record
insert 10026 record
insert 10027 record
insert 10028 record
insert 10029 record
insert 10030 record
insert 10031 record
insert 10032 record
insert 10033 record
insert 10034 record
insert 10035 record
insert 10036 record
insert 10037 record
insert 10038 record
insert 10039 record
insert 10040 record
insert 10041 record
insert 10042 record
insert 10043 record
insert 10044 record
insert 10045 record
insert 10046 record
insert 10047 record
insert 10048 record
insert 10049 record
insert 10050 record
insert 10051 record
insert 10052 record
insert 10053 record
insert 10054 record
insert 10055 record
insert 10056 record


insert 10418 record
insert 10419 record
insert 10420 record
insert 10421 record
insert 10422 record
insert 10423 record
insert 10424 record
insert 10425 record
insert 10426 record
insert 10427 record
insert 10428 record
insert 10429 record
insert 10430 record
insert 10431 record
insert 10432 record
insert 10433 record
insert 10434 record
insert 10435 record
insert 10436 record
insert 10437 record
insert 10438 record
insert 10439 record
insert 10440 record
insert 10441 record
insert 10442 record
insert 10443 record
insert 10444 record
insert 10445 record
insert 10446 record
insert 10447 record
insert 10448 record
insert 10449 record
insert 10450 record
insert 10451 record
insert 10452 record
insert 10453 record
insert 10454 record
insert 10455 record
insert 10456 record
insert 10457 record
insert 10458 record
insert 10459 record
insert 10460 record
insert 10461 record
insert 10462 record
insert 10463 record
insert 10464 record
insert 10465 record
insert 10466 record
insert 10467 record


,product_id,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
0,1,10441,13535,16974,26088,4210,39657,22028,20955,17122,33000
1,4,24852,0,0,0,0,0,0,0,0,0
2,10,18465,36011,21405,18362,48679,24841,6187,24838,24184,14947
3,23,28199,5134,2452,43789,13629,20995,16797,34126,17948,5876
4,25,39972,25340,30169,22963,22227,13646,25513,40709,11777,45007
...,...,...,...,...,...,...,...,...,...,...,...
10822,49655,27344,30391,46979,22825,43961,0,0,0,0,0
10823,49667,35469,24195,44366,43122,34969,13829,45613,27344,30391,46979
10824,49668,24852,0,0,0,0,0,0,0,0,0
10825,49680,49683,28204,4605,23734,41844,42265,4799,46979,8518,21903
